In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
from fuzzywuzzy import process

In [2]:
# load netindex dataset
nidf = pd.read_csv('./net_index/region_daily_speeds.csv')
nidf.head().T

,0,1,2,3,4
country,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan
region,Kabol,Kabol,Kabol,Kabol,Kabol
date,2010-07-15,2010-07-16,2010-07-17,2010-07-18,2010-07-19
download_kbps,226.6,270.211,313.048,360.694,447.529
upload_kbps,105.9,106.895,114.143,109.863,105.736
total_tests,10,19,21,31,35
distance_miles,244.01,238.295,237.076,234.949,236.993


In [3]:
# extract country names
country_names_to_match = list(nidf[['country']].value_counts().to_dict().keys())
country_names_to_match[:5]

[('United Kingdom',),
 ('Russian Federation',),
 ('United States',),
 ('Turkey',),
 ('Thailand',)]

In [4]:
# load gadm36 shapefile
gsf = shapefile.Reader('./gadm36_levels_shp/gadm36_1')

In [5]:
# show shapefile fields
[field[0] for field in gsf.fields[1:]]

['GID_0',
 'NAME_0',
 'GID_1',
 'NAME_1',
 'VARNAME_1',
 'NL_NAME_1',
 'TYPE_1',
 'ENGTYPE_1',
 'CC_1',
 'HASC_1']

In [6]:
# load country names from shapefile
gdf = pd.DataFrame(
    [
        [
            record[1],
            record[3],
            [
                *(record[4].split('|')),
                *(record[5].split('|'))
            ]
        ] for record in gsf.iterRecords()
    ],
    columns=['country', 'region', 'region_alt']
)
country_name_choices = list(gdf['country'].unique())
country_name_choices[:5]

['Afghanistan', 'Angola', 'Åland', 'Albania', 'Andorra']

In [7]:
# match countries and fuzzy match failed cases
ok_count = 0
fail_count = 0

for country_name in country_names_to_match:
    country_name = country_name[0]
    if gdf[gdf['country'] == country_name].shape[0] != 0:
        ok_count += 1
    else:
        print(country_name, '=>', process.extract(country_name, country_name_choices, limit=3))
        fail_count += 1

print('OK', ok_count, 'FAIL', fail_count)

Russian Federation => [('Russia', 90), ('Iran', 68), ('Spain', 54)]
Iran, Islamic Republic of => [('Iran', 90), ('Czech Republic', 86), ('Isle of Man', 86)]
Kazakstan => [('Kazakhstan', 95), ('Pakistan', 71), ('Kyrgyzstan', 63)]
Korea, Republic of => [('Democratic Republic of the Congo', 86), ('Isle of Man', 86), ('South Korea', 86)]
Moldova, Republic of => [('Moldova', 90), ('Democratic Republic of the Congo', 86), ('Isle of Man', 86)]
Syrian Arab Republic => [('Syria', 90), ('Democratic Republic of the Congo', 86), ('Czech Republic', 69)]
Palestinian Territory => [('Palestina', 80), ('Estonia', 77), ('India', 72)]
Macau => [('Macao', 80), ('Jamaica', 67), ('Malta', 60)]
Aruba => [('Barbados', 72), ('Cuba', 67), ('Paraguay', 62)]
Lao People's Democratic Republic => [('Republic of Congo', 86), ('Czech Republic', 86), ('Dominican Republic', 86)]
Gibraltar => [('Malta', 72), ('Qatar', 72), ('Iran', 68)]
Maldives => [('Mali', 68), ('Malawi', 57), ('Bangladesh', 56)]
Virgin Islands, Britis

In [8]:
# manually correct netindex dataset
nidf.replace({'country': 'Russian Federation'}, 'Russia', inplace=True)
nidf.replace({'country': 'Korea, Republic of'}, 'South Korea', inplace=True)
nidf.replace({'country': 'Kazakstan'}, 'Kazakhstan', inplace=True)
nidf.replace({'country': 'Iran, Islamic Republic of'}, 'Iran', inplace=True)
nidf.replace({'country': 'Palestinian Territory'}, 'Palestina', inplace=True)
nidf.replace({'country': 'Moldova, Republic of'}, 'Moldova', inplace=True)
nidf.replace({'country': 'Syrian Arab Republic'}, 'Syria', inplace=True)
nidf.replace({'country': 'Brunei Darussalam'}, 'Brunei', inplace=True)
nidf.replace({'country': 'Macau'}, 'Macao', inplace=True)
nidf.replace({'country': 'Netherlands Antilles'}, 'Netherlands', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Aruba'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Lao People\'s Democratic Republic'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Gibraltar'].index, inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Maldives'].index, inplace=True)
nidf.replace({'country': 'Virgin Islands, British'}, 'British Virgin Islands', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Anguilla'].index, inplace=True)
nidf.replace({'country': 'Tanzania, United Republic of'}, 'Tanzania', inplace=True)
nidf.replace({'country': 'Cote D\'Ivoire'}, 'Côte d\'Ivoire', inplace=True)
nidf.replace({'country': 'Aland Islands'}, 'Åland', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Curacao'].index, inplace=True)
nidf.replace({'country': 'Libyan Arab Jamahiriya'}, 'Libya', inplace=True)
nidf.drop(nidf.loc[nidf['country'] == 'Saint Martin'].index, inplace=True)

In [9]:
# match again to validate
ok_count = 0
fail_count = 0

for country in list(nidf[['country']].value_counts().to_dict().keys()):
    country = country[0]
    if gdf[gdf['country'] == country].shape[0] != 0:
        ok_count += 1
    else:
        print(country)
        fail_count += 1
        
print('OK', ok_count, 'FAIL', fail_count)

OK 181 FAIL 0


In [10]:
# extract country and region names
# NOTE THAT all country names are valid now
countryregion_names_to_match = list(nidf[['country', 'region']].value_counts().to_dict().keys())
countryregion_names_to_match[:5]

[('Norway', 'Vest-Agder'),
 ('Turkey', 'Ankara'),
 ('United Kingdom', 'North Tyneside'),
 ('United Kingdom', 'North Somerset'),
 ('Indonesia', 'Yogyakarta')]

In [11]:
# load region names of each country from shapefile
countryregion_name_choices = dict(zip(
    country_name_choices,
    [gdf[gdf['country'] == country_name]['region'].to_list() for country_name in country_name_choices]
))
countryregion_name_choices['China'][:5]

['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu']

In [12]:
# load region alt names of each country from shapefile
countryregion_alt_name_choices = dict(zip(
    country_name_choices,
    [gdf[gdf['country'] == country_name]['region_alt'].to_list() for country_name in country_name_choices]
))
countryregion_alt_name_choices['China'][:5]

[['Ānhuī', '安徽', '安徽'],
 ['Běijīng', '北京', '北京'],
 ['Chóngqìng', '重慶', '重庆'],
 ['Fújiàn', '福建'],
 ['Gānsù', '甘肅', '甘肃']]

In [13]:
ok_count = 0
alt_count = 0
fm_count = 0
fail_count = 0

print(len(countryregion_names_to_match))

for countryregion_name in countryregion_names_to_match:
    country_name = countryregion_name[0]
    region_name = countryregion_name[1]
    
    if region_name in countryregion_name_choices[country_name] or region_name == r'\N':
        ok_count += 1
    else:
        # check alt names
        alt_flag = False
        for i in range(len(countryregion_alt_name_choices[country_name])):
            if region_name in countryregion_alt_name_choices[country_name][i]:
                nidf.loc[(nidf['country'] == country_name) & (nidf['region'] == region_name), 'region'] = countryregion_name_choices[country_name][i]
                alt_count += 1
                alt_flag = True
                break
        # fuzzy match
        fm_flag = False
        if not alt_flag:
            fm_results = process.extractOne(region_name, countryregion_name_choices[country_name])
            if fm_results[1] > 90:
                nidf.loc[(nidf['country'] == country_name) & (nidf['region'] == region_name), 'region'] = fm_results[0]
                fm_count += 1
                fm_flag = True
        # fallback
        if not alt_flag and not fm_flag:
            nidf.drop(nidf.loc[(nidf['country'] == country_name) & (nidf['region'] == region_name)].index, inplace=True)
            fail_count += 1

print('OK', ok_count, 'ALT', alt_count, 'FM', fm_count, 'FAIL', fail_count)

1861
OK 1025 ALT 140 FM 110 FAIL 586


In [14]:
nidf.to_csv('net_index/region_daily_speeds_name_matched.csv', index=False)